# Searching for Be stars in binaries with compact objects
## Catalog cross-matching
### Caden Gobat, The George Washington University

Some conventions/names:
- The first letter of `astropy.table` object variable names are capitalized
- First letters of `pandas.DataFrame` names are lowercase
- CXO, CSC = Chandra X-ray Observatory, Chandra Source Catalog
- XMM, SSC = XMM-Newton, Serendipitous Source Catalog
- XRT, XPS = (Swift) X-ray Telescope, XRT Point Source (catalog)
- BeSS = Be Star Spectra database

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, astropy.units as u, pickle, seaborn as sns
from mpl_toolkits import mplot3d
from scipy import stats
from PyAstronomy import pyasl
from astropy.io import fits, ascii as ascii_io
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad

from tools import hms_to_dec, dms_to_dec

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


### Initialize ViZier query instance

In [8]:
v = Vizier(columns=['all', '_RAJ2000', '_DEJ2000', "_r"])
v.ROW_LIMIT = -1 # no row limit

### Load the [Be star database](http://basebe.obspm.fr/basebe/)

In [9]:
BeSS = ascii_io.read("./catalogs/BeSS_catalog.csv", format="csv", header_start=0, data_start=1)
BeSS["_RAJ2000"] = [hms_to_dec(hms) for hms in BeSS["RA"]]*u.degree
BeSS["_DEJ2000"] = [dms_to_dec(dms) for dms in BeSS["DEC"]]*u.degree
BeSS.info

&lt;Table length=2264&gt;
         name          dtype  unit    class     n_bad
--------------------- ------- ---- ------------ -----
                    #   int32            Column     0
              Be star   str26            Column     0
             Category   str19            Column     0
                   RA   str11            Column     0
                  DEC   str12            Column     0
                    V float64      MaskedColumn   310
                 Type   str10      MaskedColumn   110
                vsini   int32      MaskedColumn  1648
Nb of spectra in BeSS   int32            Column     0
             _RAJ2000 float64  deg       Column     0
             _DEJ2000 float64  deg       Column     0

### Crossmatch with Gaia data

Create `gaia_df`, a `pandas.DataFrame` containing the single closest match (within $1^{\prime\prime}$) in the Gaia catalog for every entry in the BeSS database.

In [100]:
Gaia_BeSS_1as = v.query_region(BeSS, catalog="I/345/gaia2", radius="1s")
Gaia_matched = Gaia_BeSS_1as[0]["_q","_r","DR2Name","pmRA","pmDE","epsi","sepsi","Gmag","BPmag","RPmag","AG","RV","GLON","GLAT","Teff","Lum","Rad","Plx"]

gaia_df = Gaia_matched.to_pandas()
for i in BeSS["#"]:
    instances = len(gaia_df[gaia_df["_q"]==i])
    if instances == 0:
        pass
    elif instances >= 2:
        dupeframe = gaia_df[gaia_df["_q"]==i][["_q","_r"]]
        gaia_df.drop(dupeframe[dupeframe["_r"]>min(dupeframe["_r"])].index, inplace=True)

gaia_df.dropna(axis=0, subset=["_r"], inplace=True)
gaia_df["Gflux"] = (2835.09*10**(gaia_df["Gmag"]/(-2.5)))/(3.34e4*6246.77**2) # Jy -> erg/cm^2/s/Ang

assert(len(gaia_df) == len(np.unique(gaia_df["_q"]))),"Something went wrong..."

### HMXB crossmatch

- 2CXO J210335.6+454505 with 92%/100% is a HMXB in our TD
- 2CXO J213930.6+565910 with 83%/99% is a HMXB from Simbad, but not in our TD
- 2CXO J112057.1-615500 with 96%/100% is a HMXB from Liu+06 in our TD
- 2CXO J223920.8+611626 with 81%/100% is a HMXB from Liu+06 in our TD
- 2CXO J174445.7-271344 #  not significant with 45%/70% is a HMXB in our TD, but a Be Star in Simbad
- 2CXO J024031.6+611345 #  not significant with 58%/95% is a HMXB from Liu+06
- 2CXO J113106.9-625648 #  not significant with 36%/35% is not in our TD, is a HMXB from Simbad
- 2CXO J130247.6-635008 #  not significant with 53%/56% is not in our TD, is a HMXB from Simbad, 
- 2CXO J134632.5-625523 #  not significant with 44%/47% is not in our TD, is a Be Star from Simbad

In [21]:
HMXBs_BeSS_1as = v.query_region(BeSS, catalog="J/A+A/455/1165/table1", radius="1s")

### XMM, Chandra, and Swift/XRT catalogs

| Band      | Energy bin range |
|-----------|------------------|
| XMM Flux1 | 0.2 - 0.5 keV    |
| XMM Flux2 | 0.5 - 1.0 keV    |
| XMM Flux3 | 1.0 - 2.0 keV    |
| XMM Flux4 | 2.0- 4.5 keV     |
| XMM Flux5 | 4.5 - 12.0 keV   |
| XMM Flux8 | 0.2 - 12.0 keV   |
| XMM Flux9 | 0.5 - 4.5 keV    |
| CXO Fluxu | 0.2 - 0.5 keV    |
| CXO Fluxs | 0.5 - 4.2 keV    |
| CXO Fluxm | 1.2 - 2.0 keV    |
| CXO Fluxh | 2.0 - 7.0 keV    |
| CXO Fluxb | 0.5 - 7.0 keV    |

Load the full catalogs from locally stored copies. Crossmatch from vizier remotely.

In [22]:
with open("./catalogs/xmm4dr9s.pkl","rb") as file:
    XMM = pickle.load(file)
with open("./catalogs/csc2master.pkl","rb") as file:
    CSC = pickle.load(file)
with open("./catalogs/2sxpscle.pkl","rb") as file:
    XRT = pickle.load(file)

cat_ids = ["IX/59/xmm4dr9s","IX/57/csc2master","IX/58/2sxpscle"]
Xmatch = [None, None, None]
for i in range(len(cat_ids)):
    Xmatch[i] = v.query_region(BeSS, catalog=cat_ids[i], radius="1s")
    print("Successfully matched BeSS with",cat_ids[i])
[XMM_xmatch, CSC_xmatch, XRT_xmatch] = Xmatch

Successfully matched BeSS with IX/59/xmm4dr9s
Successfully matched BeSS with IX/57/csc2master
Successfully matched BeSS with IX/58/2sxpscle


In [94]:
Chandra_sources = CSC_xmatch[0]["_q","_2CXO","Fluxb","Fluxh","Fluxm","Fluxs","Fluxu",
                                "VaKSb","VaKPb","VaGLb","Vib"]
chandra_bands = {"b":(0.5,7), "h":(2,7), "m":(1.2,2), "s":(0.5,1.2), "u":(0.2,0.5)}

XMM_sources = XMM_xmatch[0]["_q","_4XMM","Flux1","Flux2","Flux3","Flux4","Flux5","Flux8","Flux9",
                            "Cst","Fvar","e_Fvar","V"]
xmm_bands = dict(zip(list("1234589"),[(0.2,0.5),(0.5,1.0),(1.0,2.0),(2.0,4.5),(4.5,12.0),(0.2,12.0),(0.5,4.5)]))

### Compile a master catalog

In [121]:
objs = pd.merge(gaia_df, HMXBs_BeSS_1as[0].to_pandas(), on="_q", how="outer")
xray = pd.merge(XMM_sources.to_pandas(), Chandra_sources.to_pandas(), on="_q", how="outer")

master = pd.merge(BeSS.to_pandas(), pd.merge(objs, xray, on="_q", how="outer"), left_on="#", right_on="_q", how="outer")

In [122]:
fluxnums = [str(i) for i in [1,2,3,4,5]]
for i in range(len(fluxnums)-1):
    A = master[f"Flux{fluxnums[i+1]}"]
    B = master[f"Flux{fluxnums[i]}"]
    master[f"HR_{fluxnums[i+1]}{fluxnums[i]}"] = (A-B)/(A+B)
    
fluxchars = list("usmh")
for i in range(len(fluxchars)-1):
    A = master[f"Flux{fluxchars[i+1]}"]
    B = master[f"Flux{fluxchars[i]}"]
    master[f"HR_{fluxchars[i+1]}{fluxchars[i]}"] = (A-B)/(A+B)

for i in master.index:
    xmm_flux = master.loc[i,"Flux8"]
    cxo_flux = master.loc[i,"Fluxb"]
    G_flux = master.loc[i,"Gflux"]
    if pd.isna(G_flux):
        continue
    xmm_B_ox = np.log10(xmm_flux/G_flux)/np.log10(6.1*2.42e17/(3e18/6246.77))
    cxo_B_ox = np.log10(cxo_flux/G_flux)/np.log10(3.75*2.42e17/(3e18/6246.77))
    if pd.isna(xmm_B_ox):
        master.loc[i,"B_ox"] = cxo_B_ox
    elif pd.isna(cxo_B_ox):
        master.loc[i,"B_ox"] = xmm_B_ox
    else:
        master.loc[i,"B_ox"] = np.mean([xmm_B_ox, cxo_B_ox])

In [123]:
for i in master.index:
    flags = []
    if not pd.isna(master.loc[i,"Be star"]):
        flags.append("BeSS")
    if not pd.isna(master.loc[i,"DR2Name"]):
        flags.append("Gaia")
    if not pd.isna(master.loc[i,"Name"]):
        flags.append("HMXB")
    if not pd.isna(master.loc[i,"_4XMM"]):
        flags.append("XMM")
    if not pd.isna(master.loc[i,"_2CXO"]):
        flags.append("Chandra")
    master.loc[i,"flags"] = ",".join(flags)

In [124]:
master.drop(['RA','DEC','_r_x','_RAJ2000_y','_DEJ2000_y','_r_y','RAJ2000','DEJ2000','GLON_y','GLAT_y',"u_Name2"], axis=1, inplace=True)
master.rename(columns={"V_x":"V","Type_x":"SpecType",'_RAJ2000_x':"_RAJ2000",'_DEJ2000_x':"_DEJ2000","GLON_x":"GLON","GLAT_x":"GLAT","Type_y":"Type","V_y":"Vflag"}, inplace=True)

In [125]:
master.to_csv("./catalogs/BeSS+Gaia+HMXBs+CSC+XMM.csv", index=False)